In [282]:
import os
import pandas as pd
import cv2

deleted_range = []
right_side = []

def render_ui_frame_id(frame, frame_id, frame_total):
    cv2.rectangle(frame, (10, 5), (400, 35), (0,0,0), -1)
    cv2.putText(frame, f'{frame_total} : {frame_id}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

def render_index(frame,current_idx,total_idx):
    cv2.putText(frame, f'{current_idx}/{total_idx}', (10, 350), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

def show_vdo(_ep_id):
    session_id = vdo_file[:16] + r'_scaled'
    episode_path = _ep_id
    print(episode_path)
    df = pd.read_csv(episode_path, usecols=['start', 'end'])
    df_list = df.values.tolist()
    video_path = os.path.join(os.path.join(os.getcwd(), 'Dataset'), f"{session_id}.MOV")
    # print(video_path)
    capture_handler = cv2.VideoCapture(video_path)
    if not capture_handler.isOpened():
        print(f"Error: Unable to open the video file {video_path}")
        exit()
    # Get the total frame of the video%
    frame_total = int(capture_handler.get(cv2.CAP_PROP_FRAME_COUNT))
    # Define the window name, also use it as tag for using in CV
    window_name = f"{session_id} - mICE Video Marker"
    # Create a window
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    frame_id_current = 0
    saved_frame = None
    flag_pause = True
    is_on_seeking = False
    cnt_episode_frame = 0
    flag = True
    total_index = len(df_list)
    while flag ==True:
        try:
        # Check if the video is paused, if so, then keep playing the current frame
            if is_on_seeking:
                 if flag_is_set_marker_founded:
                    is_on_seeking = False
                    flag_is_set_marker_founded = False
                    flag_pause = True
                    saved_frame = None
                    capture_handler.set(cv2.CAP_PROP_POS_FRAMES, frame_id_current)
            if flag_pause:
                if saved_frame is None:
                    ret, saved_frame = capture_handler.read()
                frame = saved_frame.copy()
            else:
                ret, frame = capture_handler.read()
            frame_id_current = int(capture_handler.get(cv2.CAP_PROP_POS_FRAMES)) - 1
            # Render frame info
            render_ui_frame_id(frame, frame_id_current, frame_total)
            render_index(frame,cnt_episode_frame,total_index-1)
            # Display the frame
            cv2.imshow(window_name, frame)
            # Wait for a key press, q to exit, space to start/stop the video (keep playing current frame)
            key = cv2.waitKey(1) & 0xFF
            if key in [ord('q'),ord('Q')]:
                cv2.destroyAllWindows()
                flag = False
            elif key in [ord('R'),ord('r'),ord('d'),ord('D'),ord(' '),ord('Z'),ord('z'),ord('x'),ord('v'),ord('b'),ord('n'),ord ('m'),ord('e'),ord('E'),ord('w'),ord('W'),ord('t'),ord('T'),ord('y'),ord('Y')]:
                if key == ord(' '):
                    is_on_seeking = False
                    if flag_pause:
                        flag_pause = False
                        saved_frame = None
                    else:
                        flag_pause = True
                    frame_id_current = int(capture_handler.get(cv2.CAP_PROP_POS_FRAMES)) - 1
                if key==ord('r') or key==ord('R'):
                    cnt_episode_frame -=1
                    frame_id_current = df_list[cnt_episode_frame][0] 
                if key==ord('d') or key==ord('D'):
                    cnt_episode_frame +=1
                    frame_id_current = df_list[cnt_episode_frame][0] 
                    if cnt_episode_frame >= frame_total:
                        cnt_episode_frame = frame_total

                if key==ord('e') or key==ord('E'):
                    if(frame_id_current!=df_list[cnt_episode_frame][0]):
                        frame_id_current = df_list[cnt_episode_frame][0]
                    deleted_range.append(frame_id_current)
                    print(deleted_range[-1])
                if key==ord('w') or key==ord('W'):
                    del deleted_range[-1]
                    for i in range(0, len(deleted_range), 10):
                        print(*deleted_range[i:i+10])
                if key==ord('t') or key==ord('T'):
                    if(frame_id_current!=df_list[cnt_episode_frame][0]):
                        frame_id_current = df_list[cnt_episode_frame][0]
                    right_side.append(frame_id_current)
                    print(right_side)
                if key==ord('y') or key==ord('Y'):
                    del right_side[-1]
                    print(right_side)
                if key == ord('Z'):
                    frame_id_current = 0
                if key == ord('z'):
                    frame_id_current -= 240
                if key == ord('x'):
                    frame_id_current -= 1
                if key == ord('v'):
                    frame_id_current += 1
                if key == ord('b'):
                    frame_id_current += 240
                if key == ord('n'):
                    frame_id_current += 120
                if key == ord('m'):
                    frame_id_current += 500
                capture_handler.set(cv2.CAP_PROP_POS_FRAMES, frame_id_current)
                saved_frame = None
        except Exception as e:
            print(f"Error in {e}")
            continue
        

In [283]:
# input_dir = './g2_right_unclean'
right_dir = './g5_right_unclean'
left_dir = './g5_left_output_episodes'
cr_dir = './g5_right_clean'
left_unclean_dir = './g5_left_unclean'
left_clean_dir = './g5_left_clean'
no_cr_ep = False #ไม่มีขวา True
only_left = False 

selected_file = '0b27065b15a4300c'
for i, file in enumerate(os.listdir(right_dir)[:]):
    if file[0:15] == selected_file[0:15]:
        selected_file = file
        if not no_cr_ep:
            cr_ep_filename = os.path.join(cr_dir,r'right_'+ file[:-4]+r'_episodes.csv')
        right_ep_filename = os.path.join(right_dir, file)
        left_ep_filename = os.path.join(left_dir, r'left_'+file)
        if not no_cr_ep:
            cr_ep_filename = os.path.join(cr_dir,r'right_'+ file)
        no_right_filename = os.path.join(left_unclean_dir,r'no_right_'+file)
        print(no_right_filename)
        left_output_filename = os.path.join(left_clean_dir,r'left_'+file)

        vdo_file = file
        right_ep_df = pd.read_csv(right_ep_filename)
        left_ep_df = pd.read_csv(left_ep_filename)
        if not no_cr_ep:
            clean_right_ep_df = pd.read_csv(cr_ep_filename)
            print(f'len right:{len(right_ep_df)}, cr:{len(clean_right_ep_df)}; {len(right_ep_df)-len(clean_right_ep_df)}')
            print(f'len left:{len(left_ep_df)}; {len(left_ep_df)+(len(right_ep_df)-len(clean_right_ep_df))}')
        else:
            print(f'len right:{len(right_ep_df)}')
            print(f'len left:{len(left_ep_df)}')


./g5_left_unclean\no_right_0b27065b15a4300c_scaledDLC_mobnet_100_y79Jun20shuffle1_10000_episodes.csv


FileNotFoundError: [Errno 2] No such file or directory: './g5_left_output_episodes\\left_0b27065b15a4300c_scaledDLC_mobnet_100_y79Jun20shuffle1_10000_episodes.csv'

In [276]:
if only_left:
    no_right = left_ep_df
elif not no_cr_ep:
    all_df = right_ep_df.merge(clean_right_ep_df, on='start',how='left', indicator=True)
    # print(all_df.to_string())
    no_right = all_df[all_df["_merge"] == "left_only"].drop(columns=["_merge"]).dropna(axis='columns')
    no_right.columns = right_ep_df.columns
else:
    no_right = right_ep_df
    
no_right = pd.concat([no_right, left_ep_df]).drop_duplicates(subset=['start']).sort_values(by=['start'])
print(len(no_right))
# print(no_right.to_string())
no_right.to_csv(no_right_filename,index=False)
# no_right.to_csv(left_output_filename,index=False)


244


C:\Users\mice\AppData\Local\Temp\ipykernel_9188\1285493843.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  no_right = pd.concat([no_right, left_ep_df]).drop_duplicates(subset=['start']).sort_values(by=['start'])


In [277]:
show_vdo(os.getcwd()+no_right_filename)

c:\Users\mice\Desktop\sum_all_process./g5_left_unclean\no_right_2f9d144675ac3cd8_scaledDLC_mobnet_100_y79Jun20shuffle1_10000_episodes.csv
[200765, 98, 8846, 9365, 29777, 35777, 47052, 49902, 59584, 61153, 68830, 76358, 82284, 82758, 245820, 294762, 301999, 311742, 313598, 70190, 71686, 19287, 48991, 65192, 19287, 26038, 19287, 26038, 48991, 65192, 10981, 152019, 27551, 41303, 110163, 125967, 134068, 180825, 43817, 89834, 102502, 103164, 104140, 172055, 484, 3132, 29046, 62694, 102819, 105079, 113502, 115351, 125995, 125995, 148403, 154695, 168617, 198201, 199043, 202621, 205819, 212115, 212416, 215941, 218477, 218477, 226021, 227076, 229434, 77, 13333, 46668, 50696, 130090, 12081]
61990
65259
111869
119858
119953
120167
157538
180791
198542
Error in list index out of range
Error in list index out of range


In [278]:
print(len(deleted_range))
left_df = pd.read_csv(os.getcwd()+no_right_filename)
print(len(left_df))
left_df = left_df[~left_df['start'].isin(deleted_range)]
left_df = left_df[~left_df['start'].isin(right_side)]
print(len(left_df))
left_df.to_csv(left_output_filename,index=False)
print(f"right_side : {right_side}")
if len(right_side)!= 0:
    print(f"right_side : {right_side}")
    new_cr_path = os.getcwd() + r'\g5_new_cr' + '\\' + r'right_' + selected_file
    print(new_cr_path)
    # print(no_right.loc[no_right['start'].isin(right_side)])
    new_right_df = no_right[no_right['start'].isin(right_side)]
    print(len(clean_right_ep_df))
    new_cr = pd.concat([clean_right_ep_df, new_right_df]).drop_duplicates(subset=['start']).sort_values(by=['start'])
    print(len(new_cr))
    new_cr.to_csv(new_cr_path,index=False)

172
244
234
right_side : [200765, 98, 8846, 9365, 29777, 35777, 47052, 49902, 59584, 61153, 68830, 76358, 82284, 82758, 245820, 294762, 301999, 311742, 313598, 70190, 71686, 19287, 48991, 65192, 19287, 26038, 19287, 26038, 48991, 65192, 10981, 152019, 27551, 41303, 110163, 125967, 134068, 180825, 43817, 89834, 102502, 103164, 104140, 172055, 484, 3132, 29046, 62694, 102819, 105079, 113502, 115351, 125995, 125995, 148403, 154695, 168617, 198201, 199043, 202621, 205819, 212115, 212416, 215941, 218477, 218477, 226021, 227076, 229434, 77, 13333, 46668, 50696, 130090, 12081]
right_side : [200765, 98, 8846, 9365, 29777, 35777, 47052, 49902, 59584, 61153, 68830, 76358, 82284, 82758, 245820, 294762, 301999, 311742, 313598, 70190, 71686, 19287, 48991, 65192, 19287, 26038, 19287, 26038, 48991, 65192, 10981, 152019, 27551, 41303, 110163, 125967, 134068, 180825, 43817, 89834, 102502, 103164, 104140, 172055, 484, 3132, 29046, 62694, 102819, 105079, 113502, 115351, 125995, 125995, 148403, 154695, 16

In [279]:
show_vdo(os.getcwd() + left_output_filename)

c:\Users\mice\Desktop\sum_all_process./g5_left_clean\left_2f9d144675ac3cd8_scaledDLC_mobnet_100_y79Jun20shuffle1_10000_episodes.csv


In [280]:
episode_id = os.getcwd() + r'\g5_new_cr' + '\\' + r'right_' + selected_file
show_vdo(episode_id)

c:\Users\mice\Desktop\sum_all_process\g5_new_cr\right_2f9d144675ac3cd8_scaledDLC_mobnet_100_y79Jun20shuffle1_10000_episodes.csv
